# Introdução

Este script segue o procedimento apresentado na Cartilha GTFS do MOB 4.0 para consolidar dados de transporte público em uma estrutura consistente com a Especiicação Geral de Feeds de Transporte Público, [conforme utilizada pelo Google](https://developers.google.com/transit/gtfs?hl=pt-br). A cartilha contém instruções a respeito de como deve ser a estrutura dos dados de entrada, estrutura essa que é essencial para que o código que aqui consta se comporte como esperado.

Ao longo deste script, pontualmente, são feitas considerações gerais a respeito do funcionamento das linhas de código, mas não se entende que seja necessário conhecimento prévio de linguagem de programação. O código foi pensado de maneira que ele apenas necessita, logo no início, dos inputs do usuário. Feito isso, após processamento, o scritp irá retornar um arquivo ZIP o qual contém os arquivos GTFS básicos. Não obstante, como o código foi completamente disponibilizado, de forma transparente, sugestões de melhora e aperfeiçoamento são bem vindas e podem ser incorporadas para versões futuras.

Atualmente o código apenas suporta a construção de redes de ônnibus regulares, sobretudo porque este é o caso mais comum presente nas cidades brasileiras. No entanto, versões futuras irão incorporar outros modos.

Este código é a primeira versão publicada pelo MOB 4.0 e antecipam-se novas edições com funcionamento aprimorado, seja por nossa própria iniciativa, seja após sugestão de eventuais usuários.

# Instruções

**Agora, a ideia é que o usuário forneça os arquivos necessários para o bom funcionamento do script. Feito isso, basta processar o script e ao final ele produzirá os arquivos GTFS para o sistema de transporte público em questão.** Esta primeira versão consegue lidar com sistemas de uma única cidade. Caso haja linhas intermunicipais que sejam indispensáveis para o sistema, deve ser considerado apenas a parte do trajeto que se localiza no interior do município. Posto isso, às instruções:

- Fornecer código do IBGE para o município, conforme lista hospedada [aqui](https://www.ibge.gov.br/explica/codigos-dos-municipios.php);
- Fornecer caminho no computador para a planilha operacional que compõe a cartilha;
- Fornecer caminho no computador para o arquivo geográfico com os pontos de parada;
    - Em não havendo disponibilidade dos pontos de ônibus, o script irá assumir que há pontos de ônibus a cada 350 metros, ao longo do itinerário, por padrão. Caso se queira fornecer uma outra distância, preencher a variável espacamento com uma distância que se ache mais apropriada.
- Fornecer caminho no computador para o arquivo geográfico com os itinerários.

In [ ]:
# Substitituir None com o código do IBGE para o município
ibge_id = 3143302

# Substitituir None com o caminho para a planilha excel
planilha_operacional = '/content/planilha_gtfs.xlsx'

# Substituir None com o caminho para o arquivo geográfico
# com pontos de parada, se houver
pontos_de_parada = None#'/content/pontos_bus.kml'

# Substituir None com o caminho para o arquivo geográfico com itinerarios
itinerarios = '/content/linhas_mc.kml'

# Escolher distância em metros entre pontos de parada.
# Apenas se os pontos de parada não forem fornecidos e caso
# não se deseje utilizar o valor padrão de 350 m
espacamento = 350

# Set up

Aqui é feita a instalação e a importação das bibliotecas necessárias, assim como também são feitos alguns ajustes iniciais gerais.

In [ ]:
!pip install osmnx
!pip install h3
!pip install tobler
!pip install --no-deps geobr
!pip install git+https://github.com/remix/partridge.git

  Cloning https://github.com/remix/partridge.git to /tmp/pip-req-build-43vrq9q3
  Running command git clone --filter=blob:none --quiet https://github.com/remix/partridge.git /tmp/pip-req-build-43vrq9q3
  Resolved https://github.com/remix/partridge.git to commit 7b3d2c70281ef4f5437c3e48d786569c1702ebd8
  Preparing metadata (setup.py) ... done


In [ ]:
import calendar
import re
import datetime as dt
import requests
import urllib.request
from uuid import uuid4

import fiona
import geobr
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
import partridge as ptg
from shapely.geometry import Point, LineString
from shapely.ops import linemerge
from tobler.util import h3fy

In [ ]:
# enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
path_operations = planilha_operacional
path_stops = pontos_de_parada
spacing = espacamento
path_routes = itinerarios

# Variáveis de Base

In [ ]:
borders = geobr.read_municipality(ibge_id)

EPSG = borders.estimate_utm_crs('SIRGAS 2000').to_epsg()

In [ ]:
road_graph = ox.graph_from_polygon(
    borders.to_crs(4326).geometry.squeeze(),
    network_type='drive',
    )

road_graph = ox.project_graph(
    road_graph,
    to_crs=EPSG,
    )

# A Classe Feed

Este código usa as funcionalidades da biblioteca [Partridge](https://github.com/remix/partridge), pensada para rápida leitura e manipulação de arquivos GTFS. Nesse sentido, primeiro é gerado um objeto para armazenar os arquivos GTFS conforme eles são gerados. Aqui, optamos pela forma mais simples e funcional e rápida de implementação, mas essa primeira parte passará por refinamentos na próxima edição do Script.

In [ ]:
import urllib.request

url = 'https://s3.amazonaws.com/mobilibus-uploads/gtfs/GTFSBHTRANSCON.zip'
filename = 'base_feed.zip'

urllib.request.urlretrieve(url, filename)

('base_feed.zip', <http.client.HTTPMessage at 0x7de4d7d5fbb0>)

In [ ]:
feed = ptg.load_feed(
    'base_feed.zip',
    )

filenames = [
    p
    for p
    in dir(feed)
    if isinstance(getattr(feed, p), pd.DataFrame)
    ]

for file in filenames:
    feed.set(
        f"{file}.txt",
        ptg.utilities.empty_df(),
        )

# Leitura dos Dados

In [ ]:
def _assign_ids(df):
    number_of_ids = len(df)
    return [
        str(uuid4()).split('-')[0]
        for i
        in range(number_of_ids)
        ]


In [ ]:
def _load_operations(inpath):
    return pd.read_excel(
        'planilha_gtfs.xlsx',
        sheet_name=None,
        dtype=str
        )


def read_stops(path_stops, proj_crs):
    return (
        gpd
        .read_file(path_stops)
        .rename(columns=lambda c: c.casefold())
        .rename(
            columns={
                'name': 'stop_name',
                'description':'stop_desc'
                },
            errors='ignore',
            )
        .reindex(
            columns=[
                'stop_name',
                'stop_desc',
                'geometry',
                ]
            )
        .to_crs(proj_crs)
        .assign(stop_id=_assign_ids)
        )


def _read_routes(path, proj_crs):
    df = gpd.GeoDataFrame()

    for layer in fiona.listlayers(path):
        s = gpd.read_file(path, driver='KML', layer=layer)
        df = pd.concat([df, s], ignore_index=True)

    return (
        df
        .to_crs(proj_crs)
        .rename(columns=lambda c: c.casefold())
        .assign(
            description=lambda x: x.description.str.casefold()
            )
        .assign(
            description=lambda x: np.where(
                x.description == 'volta',
                '1',
                '0',
                ),
            shape_id=lambda x: x.name.astype(str) + '-' + x.description
            )
            .reindex(
                columns=[
                    'shape_id',
                    'geometry',
                    ]
                )
        )


def load_feed(
    path_operations=None,
    path_stops=None,
    path_routes=None,
    proj_crs=EPSG,
    ):
    ops = _load_operations(path_operations)

    stops = None
    if path_stops is not None:
        stops = read_stops(path_stops, proj_crs)

    routes = None
    if path_routes is not None:
        routes = _read_routes(path_routes, proj_crs)

    return ops, stops, routes

In [ ]:
ops, stops, routes = load_feed(
    path_operations=path_operations,
    path_routes=path_routes,
    path_stops=path_stops,
    )

# GTFS Feed: Dados Não Espaciais

In [ ]:
def _rename_operations_columns(ops):
    d = {
        'id_agencia': 'agency_id',
        'linha': 'route_id',
        'nome_line': 'route_long_name',
        'sentido': 'direction_id',
        'hora_inicio': 'start_time',
        'hora_fim': 'end_time',
        'data_inicio': 'start_date',
        'data_fim': 'end_date',
        'intervalo_minutos': 'headway_secs',
        'ciclo_viagem_medio': 'travel_time',
        }

    d = d | {dia: day.casefold() for dia, day in zip(ops.iloc[:, -7:], calendar.day_name)}

    return ops.rename(columns=d)


def _build_agency(data):
    return data.rename(
        columns={
            'id_agencia': 'agency_id',
            'nome_agencia': 'agency_name',
            'fuso_horario': 'agency_timezone',
            'site_agencia': 'agency_url',
            'linguagem_agencia': 'agency_lang'
            }
        )


def _get_service_ids(df):
    u = df.monday.replace({'1': 'U', '0': '_'})
    s = df.saturday.replace({'1': 'S', '0': '_'})
    d = df.sunday.replace({'1': 'D', '0': '_'})

    return df.assign(
        service_id=u + s + d
    )


def _get_trip_ids(df):
    trip_counts = (
        df
        .groupby([
            'route_id',
            'direction_id',
            'service_id',
        ])
        .cumcount()
        .add(1)
        .astype('str')
    )

    return (
        df
        .assign(
            direction_id=lambda x: x.direction_id.str.casefold()
            )
        .assign(
            direction_id=lambda x: np.where(x.direction_id == 'volta', '1', '0')
            )
        .assign(
            trip_id=lambda x: (
                x.route_id
                + '-'
                + x.service_id.str.replace('_', '')
                + '-'
                + x.direction_id.replace({'0': 'IDA', '1': 'VOLTA'})
                + '-'
                + trip_counts
                )
            )
        )


def _add_ids(operations):
    return (
        operations
        .pipe(_get_service_ids)
        .pipe(_get_trip_ids)
        )


def _parse_frequencies(ops, feed):
    mask = ops.headway_secs.notnull() # TODO think on more sophisticated mask
    frequencies = ops.loc[mask]
    if frequencies.empty:
        feed.set(
            f"frequencies.txt",
            ptg.utilities.empty_df()
            )
        return feed

    freqs = (
        frequencies
        .reindex(
            columns=[
                'trip_id',
                'start_time',
                'end_time',
                'headway_secs',
                ]
            )
        .assign(
            exact_times=0,
            # original data was in minutes
            headway_secs=lambda x: x.headway_secs.astype(int) * 60
            )
        )

    feed.set(
        'frequencies.txt',
        freqs,
        )

    return feed


def _parse_routes(ops, feed):
    cols = [
        'route_id',
        'agency_id',
        'route_short_name',
        'route_type',
        ]
    routes = (
        ops
        .assign(
            #TODO: allow multiple agencies
            agency_id=lambda x: x.agency_id,
            route_short_name=lambda x: x.route_id,
            route_type=3,
            )
        .reindex(
            columns=cols,
            )
        .drop_duplicates(cols)
        )

    feed.set('routes.txt', routes)

    return feed


def _parse_trips(ops, feed):
    cols = [
        'route_id',
        'service_id',
        'trip_id',
        'direction_id',
        'shape_id',
        ]

    schedule = (
        ops
        .assign(
            shape_id=lambda x: (
                x.route_id
                + '-'
                + x.direction_id
                )
            )
        .drop_duplicates(cols)
        )

    feed.set(
        'trips.txt',
        schedule.reindex(columns=cols),
        )

    return feed, schedule


def _parse_calendar(ops, feed):
    days = map(lambda s: s.casefold(), calendar.day_name)
    cols = ['service_id'] + list(days) + ['start_date', 'end_date']
    feed_calendar = (
        ops
        .drop_duplicates(cols)
        .reindex(columns=cols)
        .astype(int, errors='ignore')
        )

    feed.set('calendar.txt', feed_calendar)

    return feed


def _parse_operations(
    ops,
    feed,
    ):
    feed = _parse_frequencies(ops, feed)
    feed = _parse_routes(ops, feed)
    feed, schedule = _parse_trips(ops, feed)
    feed = _parse_calendar(ops, feed)

    return feed, schedule


def build_nonspatial_files(ops, feed):
    agency, operations = ops.values()

    feed.set(
        f"agency.txt",
        _build_agency(agency)
        )

    feed, schedule = (
        operations
        .pipe(_rename_operations_columns)
        .pipe(_add_ids)
        .pipe(_parse_operations, feed)
        )

    return feed, schedule

In [ ]:
feed, schedule = build_nonspatial_files(ops, feed)

# GTFS Feed: Arquivos Espaciais

## Pontos de parada

In [ ]:
def _name_stops(df):
    rng = np.random.default_rng(42)
    word_site = "https://www.mit.edu/~ecprice/wordlist.10000"
    response = requests.get(word_site)
    WORDS = response.content.splitlines()

    chosen_words = rng.choice(
        WORDS,
        size=len(df),
        replace=False,
        )
    return [str(w, 'utf-8').upper() for w in chosen_words]


In [ ]:
def _add_spacings(df, spacing, stops_by_route):
    # TODO: exception handling and all
    if spacing:
        # TODO: different spacings per route
        return df.assign(spacing=spacing)

    if stops_by_route:
        # TO DO: make it accept scalar: same value for all
        return (
            df
            .merge(
                stops_by_route.rename('number_stops'),
                left_on='route_id',
                right_index=True,
                )
            .pipe(_interstop_distance)
            .drop(columns='number_stops')
        )


def _break_route(row):
    route_geometry = row.geometry
    spacings = np.arange(0, route_geometry.length, row.spacing)
    route_stops = [route_geometry.interpolate(s) for s in spacings]

    if route_geometry.length - spacings[-1] > row.spacing / 2 :
        return route_stops + [Point(route_geometry.coords[-1])]

    return route_stops

In [ ]:
def _get_grid(road_graph, proj_crs):
    _, e = ox.graph_to_gdfs(road_graph)
    city_bounds = gpd.GeoSeries(
        data=e.unary_union.envelope,
        crs=proj_crs,
        )
    return h3fy(city_bounds, resolution=9)


def _select_stop_at_random(gdf):
    """
    Given a grid with a cluster of stops within each cell,
    randomly selects one of them as the representative: using
    centroids may lead them to locate in weird places, depending
    on street network topology.
    """
    #TODO: when selecting points randomly, assign
    # higher probabilities to the ones closer
    # to the centroid of the set of points
    rng = np.random.default_rng(42)
    return (
        gdf
        .geometry
        .map(
            lambda g: (
                g
                if g.geom_type == 'Point'
                else rng.choice(list(g.geoms), 1)[0]
                )
            )
        )

def _groupby_grid_cell(stops_by_hex):
    return (
        stops_by_hex
        .dissolve('hex_id')
        .reset_index()
        .assign(
            geometry=_select_stop_at_random
            )
        )


def _cluster_stops(stops, road_graph, proj_crs):
    grid = _get_grid(road_graph, proj_crs)
    return (
        stops
        .sjoin(grid)
        .reindex(columns=['hex_id', 'geometry'])
        .pipe(_groupby_grid_cell)
        )


def _stops_by_route(df, proj_crs):
    return (
        df
        .assign(
            stops=lambda x: x.apply(
                _break_route,
                axis='columns'
                )
            )
        .explode('stops', ignore_index=True)
        .set_geometry('stops')
        .drop(columns='geometry')
        .rename_geometry('geometry')
        .set_crs(proj_crs)
    )

In [ ]:
def parse_stops(stops, proj_crs):
    return (
        stops
        .reindex(
            columns=[
                'stop_id',
                'stop_name',
                'stop_desc',
                'geometry'
                ]
            )
        .to_crs(4326)
        .assign(
            stop_name=_name_stops,
            stop_lat=lambda x: x.geometry.y,
            stop_lon=lambda x: x.geometry.x,
            )
        .to_crs(proj_crs)
        )

In [ ]:
def stops_from_route(
    routes,
    road_graph,
    spacing,
    stops_by_route,
    proj_crs,
    ):
    return (
        routes
        .pipe(_add_spacings, spacing, stops_by_route)
        .pipe(_stops_by_route, proj_crs)
        .pipe(_cluster_stops, road_graph, proj_crs)
        .rename(columns={'hex_id': 'stop_id'})
        .pipe(parse_stops, proj_crs)
        )


In [ ]:
def build_stops(
    feed,
    routes,
    stops=None,
    road_graph=road_graph,
    spacing=350,
    stops_by_route=None,
    proj_crs=EPSG,
    ):
    if stops is not None:
        feed.set(
            'stops.txt',
            parse_stops(stops, proj_crs).drop(columns='geometry')
            )
        return feed

    stops = stops_from_route(
        routes,
        road_graph,
        spacing,
        stops_by_route,
        proj_crs,
        )
    feed.set(
        'stops.txt',
        stops.drop(columns='geometry')
        )
    return feed

In [ ]:
feed = build_stops(
    feed,
    routes,
    stops,
    spacing=spacing,
    )

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:1295: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


# Horários de Parada

In [ ]:
def _route_surroundings(routes, search_range):
    # Future sjoin looses route geometry info,
    # which will be needed, thus the backup
    return routes.assign(
        geometry_backup=lambda x: x.geometry,
        geometry=lambda x: x.buffer(search_range),
        )


def _assign_stops_to_routes(routes, stops, search_range, epsg):
    return (
        stops
        .to_crs(epsg)
        .reindex(
            columns=[
                'stop_id',
                'geometry'
                ]
            )
        .sjoin(
            _route_surroundings(
                routes.to_crs(epsg),
                search_range,
                )
            )
        .drop(columns='index_right')
        )


def _distance_along_route(df):
    stops = df.geometry
    routes = df.geometry_backup
    dist_traveled = [
        int(round(r.project(s))) for r, s in zip(routes, stops)
        ]

    return dist_traveled


def _sequentiate_stops(df):
    return (
        df
        .groupby('trip_id')
        .cumcount()
        .add(1)
        )


def _set_first_distance_to_zero(df):
    return (
        df
        .groupby('trip_id')
        .shape_dist_traveled
        .transform(
            lambda t: [0] + t[1:].tolist()
            )
        )



def _stops_along_route(df):
    return (
        df
        .assign(
            shape_dist_traveled=_distance_along_route,
            )
        .sort_values(['trip_id', 'shape_dist_traveled'])
        .assign(
            shape_dist_traveled=_set_first_distance_to_zero,
            )
        .assign(
            stop_sequence=_sequentiate_stops,
            )
        .sort_values(['trip_id', 'stop_sequence'])
        )


In [ ]:
def _pct_dist_traveled(df):
    return (
        df
        .groupby('trip_id')
        .shape_dist_traveled
        .transform(
            lambda t: t / t.max()
            )
        )


def _arrival_times(df):
    stop_times = (
        df
        .travel_time
        .astype(int)
        .mul(60)
        .mul(
            _pct_dist_traveled(df)
            )
        )

    stop_times = np.where(
        df.end_time.isnull() | df.headway_secs.isnull(),
        stop_times + df.start_time.map(gtfs_time_to_seconds),
        stop_times
        )

    return [seconds_to_gtfs_time(t) for t in stop_times]

In [ ]:
def seconds_to_gtfs_time(total_seconds):
    if np.isnan(total_seconds):
        return total_seconds  # TODO: What to do here?
    minutes, seconds = divmod(total_seconds, 60)
    hours, minutes = divmod(minutes, 60)
    time = list(map(lambda x: str(x).rjust(2, '0'), [int(hours), int(minutes), int(seconds)]))
    return f'{time[0]}:{time[1]}:{time[2]}'


def gtfs_time_to_seconds(gtfs_time):
    h, m, s = gtfs_time.split(':')
    return int(h) * 3_600 + int(m) * 60 + int(s)


def _get_stop_times(df):
    return (
        df
        .assign(
            arrival_time=_arrival_times,
            departure_time=lambda x: x.arrival_time,
            )
    )

In [ ]:
def build_stop_times(
    routes,
    stops,
    feed,
    schedule,
    search_range=10,
    epsg=EPSG,
    ):
    stops = gpd.GeoDataFrame(
        stops,
          crs=4326,
          geometry=gpd.points_from_xy(
              stops.stop_lon,
              stops.stop_lat,
              )
          )

    stop_times = (
        _assign_stops_to_routes(routes, stops, search_range, epsg)
        .pipe(
            lambda gdf: pd.merge(
                gdf,
                feed.trips,
                )
            )
        .sort_values('shape_id')
        .reindex(
            columns=[
                'trip_id',
                'stop_id',
                'route_id',
                'direction_id',
                'shape_id',
                'geometry_backup',
                'geometry',
                ]
            )
        .pipe(_stops_along_route)
        .merge(
            schedule.reindex(
                columns=[
                    'trip_id',
                    'start_time',
                    'end_time',
                    'headway_secs',
                    'travel_time',
                    ]
                )
            )
        .pipe(_get_stop_times)
        .assign(timepoint=0)
        .reindex(
            columns=[
                'trip_id',
                'stop_sequence',
                'stop_id',
                'arrival_time',
                'departure_time',
                'shape_dist_traveled',
                'timepoint',
                ]
            )
        )

    feed.set(
        'stop_times.txt',
        stop_times
        )

    return feed

In [ ]:
feed = build_stop_times(
    routes,
    feed.stops,
    feed,
    schedule,
    )

# Shapes

Enfim, os traçados existentes são convertidos para o formato compatível com GTFS.

In [ ]:
def build_shape_txt(routes, feed):
    shapes = (
        routes
        .to_crs(4326)
        .assign(
            route_points=lambda x: x.geometry.map(
                lambda g: [Point(p) for p in g.coords]
            ),
        )
        .explode(column='route_points', ignore_index=False)
        .reset_index()
        .set_geometry('route_points')
        .assign(
            shape_pt_lat=lambda x: x.geometry.y,
            shape_pt_lon=lambda x: x.geometry.x,
            shape_pt_sequence=lambda x: x.groupby('index').cumcount(),
        )
        .drop(columns=['geometry', 'route_points'])
    )

    feed.set(
        'shapes.txt',
        shapes
        )

    return feed

In [ ]:
feed = build_shape_txt(routes, feed)

In [ ]:
feed.stop_times

,trip_id,stop_sequence,stop_id,arrival_time,departure_time,shape_dist_traveled,timepoint


# Output

In [ ]:
ptg.writers.write_feed_dangerously(
    feed,
    outpath=f'gtfs.zip'
)